#Library

In [ ]:
import pandas as pd 
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from copy import deepcopy
import regex as re
from tensorflow.python.keras.preprocessing.text import Tokenizer 
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import gensim

# Load CSV

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive/')

In [ ]:
# ga bisa:(
# df = pd.read_csv('/content/drive/Shareddrives/PowerPuffGirls/NLP/')

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving IMDB Dataset.csv to IMDB Dataset.csv


In [ ]:
df = pd.read_csv('IMDB Dataset.csv')

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [ ]:
df.shape

(50000, 2)

In [ ]:
df_original = deepcopy(df)

In [ ]:
df_original.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Spelling Correction


In [ ]:
df['review'][1304]

"This is one of the best movies. It is one of my favorites. A movie with good acting. The story is very sensitive and touching. Good camera work also.<br /><br />The names of the actresses and actors are not at the top of the American Star list. However, they give equal or better performances than the top of the list.<br /><br />It is such a pleasure to see a movie about true love, romance, friendship without having to endure watching someone having to kick-box their way to save the world.<br /><br />If you don't like this movie then you have no heart or feelings. Then go watch a sports movie. There is no killing or horror here. See the movie. It is a must. TH"

In [ ]:
def remove_tags(string):
    result = re.sub("<.*?()>|[^a-zA-Z1-9.']|[.]+",' ',string)
    return result

df['review_split'] = df['review'].apply(lambda x: remove_tags(x.lower()).split())

print(df.shape)
df.head()

(50000, 3)


,review,sentiment,review_split
0,One of the other reviewers has mentioned that ...,positive,"[one, of, the, other, reviewers, has, mentione..."
1,A wonderful little production. <br /><br />The...,positive,"[a, wonderful, little, production, the, filmin..."
2,I thought this was a wonderful way to spend ti...,positive,"[i, thought, this, was, a, wonderful, way, to,..."
3,Basically there's a family where a little boy ...,negative,"[basically, there's, a, family, where, a, litt..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[petter, mattei's, love, in, the, time, of, mo..."


In [ ]:
reviews = []
for i in range (df.shape[0]):
  reviews += df['review_split'][i]

print(len(reviews))
# reviews[1304]

unique = np.unique(np.array(reviews))

print(len(unique))


11561340
120922


In [ ]:
unique[10000]

'ambushing'

In [ ]:
# # characters -> int
# vocab_dict = {}
# count = 0
# for word in unique:
#   for character in word:
#       if character not in vocab_dict:
#           vocab_dict[character] = count
#           count += 1

# # Add special tokens
# codes = ['<PAD>','<EOS>','<GO>']
# for code in codes:
#     vocab_dict[code] = count
#     count += 1

# sorted(vocab_dict)

["'",
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '<EOS>',
 '<GO>',
 '<PAD>',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [ ]:
# # Create another dictionary to convert integers to their respective characters
# int_to_vocab = {}
# for character, value in vocab_dict.items():
#     int_to_vocab[value] = character

In [ ]:
# # Convert sentences to integers
# int_sentences = []

# for sentence in reviews:
#     int_sentence = []
#     for word in sentence:
#       int_words = []
#       for character in word:
#         int_words.append(vocab_dict[character])
#       int_sentence.append(int_words)
#     int_sentences.append(int_sentence)


# print(int_sentences)

In [ ]:
# len(reviews[0][0])

# reviews[0][0]

'o'

In [ ]:
# # Method to Relocate, remove, or add characters to create spelling mistakes
# letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
#            'n','o','p','q','r','s','t','u','v','w','x','y','z',]

# def noise_maker(sentence, threshold):   
    
#     noisy_sentence = []
#     i = 0
#     while i < len(sentence):
#         random = np.random.uniform(0,1,1)
#         # Most characters will be correct since the threshold value is high
#         if random < threshold:
#             noisy_sentence.append(sentence[i])
#         else:
#             new_random = np.random.uniform(0,1,1)
#             # ~33% chance characters will swap locations
#             if new_random > 0.67:
#                 if i == (len(sentence) - 1):
#                     # If last character in sentence, it will not be typed
#                     continue
#                 else:
#                     # if any other character, swap order with following character
#                     noisy_sentence.append(sentence[i+1])
#                     noisy_sentence.append(sentence[i])
#                     i += 1
#             # ~33% chance an extra lower case letter will be added to the sentence
#             elif new_random < 0.33:
#                 random_letter = np.random.choice(letters, 1)[0]
#                 noisy_sentence.append(vocab_to_int[random_letter])
#                 noisy_sentence.append(sentence[i])
#             # ~33% chance a character will not be typed
#             else:
#                 pass     
#         i += 1
#     return noisy_sentence

In [ ]:
# import random
# #thresh - 0 to 1
# def gen_gibberish(line,thresh=0.2):
#     times = int(random.randrange(1,len(line)) * thresh)
#     '''
#     Types of replacement:
#         1.Delete random character.
#         2.Add random character.
#         3.Replace a character.
#         4.Combination?
#     '''
#     while times!=0:
#         # try to gen noise length times...
#         times-=1
#         val = random.randrange(0,10)
#         if val <= 5:
#             #get random index
#             val = random.randrange(0,10)
#             index = random.randrange(2,len(line))
#             if val <= 3 :
#                 #delete character
#                 line = line[:index]+line[index+1:]
#             else:
#                 #add character
#                 insert_index = random.randrange(0,len(char_set))
#                 line = line[:index] + char_set[insert_index] + line[index:]
#         else:
#             index = random.randrange(0,len(char_set))
#             replace_index = random.randrange(2,len(line))
#             line = line[:replace_index] + char_set[index] + line[replace_index+1:]
#     return line

# sample = reviews[5][0]
# gib = gen_gibberish(sample)


# gib

In [ ]:
# def edits1(word):
#     "All edits that are one edit away from `word`."
#     letters    = 'abcdefghijklmnopqrstuvwxyz'
#     splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
#     deletes    = [L + R[1:]               for L, R in splits if R]
#     transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
#     replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
#     inserts    = [L + c + R               for L, R in splits for c in letters]
#     return set(deletes + transposes + replaces + inserts)


# print(len(edits1("i")))

# input_texts = []
# target_texts = []
# # REPEAT_FACTOR = 10

# for word in unique[:1000]:
#   if len(word)>1:
#     output_text = word
#     input_text = edits1(word)
#     for inp in input_text: 
#       target_texts.append(word)
#       input_texts.append(inp)

# print(len(output_text))
# print(len(input_text))


# # lines = reviews[0]
# # SKIP = int(len(lines)*0.65)


# # for line in lines[SKIP:]:
# #     if len(line)>1:
# #         output_text = '\t' + line + '\n'
# #         for _ in range(REPEAT_FACTOR):
# #             input_text = get_gibberish(line)
# #             input_texts.append(input_text)
# #             target_texts.append(output_text)
# # print("LEN OF SAMPLES:",len(input_texts))

# # input_texts
# # print(input_texts)

78
10
550


In [ ]:
# print(len(target_texts))
# print(len(input_texts))

418434
418434


In [ ]:
output_text

"'cheaters'"

In [ ]:
from keras.models import Model
from keras.layers import Input,LSTM,Dense


char_set = list(" abcdefghijklmnopqrstuvwxyz0123456789")

batch_size = 128
epochs = 1000
latent_dim = 256

num_enc_tokens = len(char_set)
num_dec_tokens = len(char_set) + 2 # includes \n \t
encoder_inputs = Input(shape=(None,num_enc_tokens))
encoder = LSTM(latent_dim,return_state=True)
encoder_outputs , state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h,state_c]

decoder_inputs = Input(shape=(None,num_dec_tokens))
decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_ouputs,_,_ = decoder_lstm(decoder_inputs,initial_state = encoder_states)

decoder_dense = Dense(num_dec_tokens, activation='softmax')
decoder_ouputs = decoder_dense(decoder_ouputs)

model = Model([encoder_inputs,decoder_inputs],decoder_ouputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 37)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 39)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        301056      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [ ]:
max_enc_len = max([len(x) for x in input_texts])
max_dec_len = max([len(x) for x in target_texts])
print("Max Enc Len:",max_enc_len)
print("Max Dec Len:",max_dec_len)


#filling in the enc,dec datas
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[ i , t , char2int[char] ] = 1
    for t,char in enumerate(target_text):
        decoder_input_data[ i, t , char2int[char] ] = 1
        if t > 0 :
            decoder_target_data[ i , t-1 , char2int[char] ] = 1
print("COMPLETED...")    

Max Enc Len: 28
Max Dec Len: 27


In [ ]:
num_samples = len(input_texts)
encoder_input_data = np.zeros( (num_samples , max_enc_len , len(char_set)),dtype='float32' )
decoder_input_data = np.zeros( (num_samples , max_dec_len , len(char_set)+2),dtype='float32' )
decoder_target_data = np.zeros( (num_samples , max_dec_len , len(char_set)+2),dtype='float32' )
print("CREATED ZERO VECTORS")


CREATED ZERO VECTORS


In [ ]:
# h=model.fit([encoder_input_data,decoder_input_data],decoder_target_data
#          ,epochs = epochs,
#           batch_size = batch_size,
#           validation_split = 0.2
#          )
# model.save('s2s.h5')

In [ ]:
import re
from collections import Counter

def words(text): 
  result = []
  for sentence in text:
    for word in sentence:
      result += re.findall(r'\w+', word)
  return result

WORDS = Counter(words(reviews))

def P(word, N=sum(WORDS.values())): 
    return WORDS[word] / N

def correction(word): 
    return max(candidates(word), key=P)

def known(words): 
    return set(w for w in words if w in WORDS)

def candidates(word): 
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
spelling_correction = [] 
for i in unique : 
  for j in i : 
    spelling_correction.append(correction(j))



In [35]:
def correct_array(arr):
  res = []
  for word in arr:
    res.append(correction(word))

# Sentiment Classification

# Preprocessing

In [ ]:
df['sentiment_binary'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

## Remove tags and split into array

In [ ]:
def remove_tags(string):
    result = re.sub("<.*?>|[^a-zA-Z1-9.']",' ',string)
    return result

df['review_split'] = df['review'].apply(lambda x: remove_tags(x.lower()).split())
df['review_split'] = df['review_split'].apply(lambda x: correct_array(x))



df.head()

## Stop words removal 

In [ ]:
# ! python -m nltk.downloader stopwords

In [ ]:
def stop_words_removal(text) : 
  stops = set(stopwords.words("english"))
  return [word for word in text if word not in (stops)]

df['review_wo_stop_words'] = df['review_split'].apply(stop_words_removal)
# test = df['review'].apply(stop_words_removal)
print(df['review_split'][0])
print(df['review_wo_stop_words'][0])

['O', 'n', 'e', ' ', 'f', ' ', 'h', 'e', ' ', 'h', 'e', 'r', ' ', 'r', 'e', 'v', 'e', 'w', 'e', 'r', ' ', 'h', ' ', 'e', 'n', 'n', 'e', ' ', 'h', ' ', 'f', 'e', 'r', ' ', 'w', 'c', 'h', 'n', 'g', ' ', 'j', 'u', ' ', '1', ' ', 'O', 'z', ' ', 'e', 'p', 'e', ' ', 'u', "'", 'l', 'l', ' ', 'b', 'e', ' ', 'h', 'k', 'e', '.', ' ', 'T', 'h', 'e', ' ', 'r', 'e', ' ', 'r', 'g', 'h', ',', ' ', ' ', 'h', ' ', ' ', 'e', 'x', 'c', 'l', ' ', 'w', 'h', ' ', 'h', 'p', 'p', 'e', 'n', 'e', ' ', 'w', 'h', ' ', 'e', '.', '<', 'b', 'r', ' ', '/', '>', '<', 'b', 'r', ' ', '/', '>', 'T', 'h', 'e', ' ', 'f', 'r', ' ', 'h', 'n', 'g', ' ', 'h', ' ', 'r', 'u', 'c', 'k', ' ', 'e', ' ', 'b', 'u', ' ', 'O', 'z', ' ', 'w', ' ', ' ', 'b', 'r', 'u', 'l', ' ', 'n', ' ', 'u', 'n', 'f', 'l', 'n', 'c', 'h', 'n', 'g', ' ', 'c', 'e', 'n', 'e', ' ', 'f', ' ', 'v', 'l', 'e', 'n', 'c', 'e', ',', ' ', 'w', 'h', 'c', 'h', ' ', 'e', ' ', 'n', ' ', 'r', 'g', 'h', ' ', 'f', 'r', ' ', 'h', 'e', ' ', 'w', 'r', ' ', 'G', 'O', '.', ' ',

In [ ]:
df.head()

,review,sentiment,sentiment_binary,review_split,review_wo_stop_words
0,One of the other reviewers has mentioned that ...,positive,1,"[One, of, the, other, reviewers, has, mentione...","[One, reviewers, mentioned, watching, 1, Oz, e..."
1,A wonderful little production. <br /><br />The...,positive,1,"[A, wonderful, little, production., The, filmi...","[A, wonderful, little, production., The, filmi..."
2,I thought this was a wonderful way to spend ti...,positive,1,"[I, thought, this, was, a, wonderful, way, to,...","[I, thought, wonderful, way, spend, time, hot,..."
3,Basically there's a family where a little boy ...,negative,0,"[Basically, there's, a, family, where, a, litt...","[Basically, there's, family, little, boy, Jake..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,"[Petter, Mattei's, Love, in, the, Time, of, Mo...","[Petter, Mattei's, Love, Time, Money, visually..."


## Lemmatization

In [ ]:
# ! python -m nltk.downloader wordnet

In [ ]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text] 

df['review_lemmatize'] = df['review_wo_stop_words'].apply(lemmatize_text)
print(df['review_wo_stop_words'][0])
print(df['review_lemmatize'][0])

['One', 'reviewers', 'mentioned', 'watching', '1', 'Oz', 'episode', 'hooked.', 'They', 'right', 'exactly', 'happened', 'me.', 'The', 'first', 'thing', 'struck', 'Oz', 'brutality', 'unflinching', 'scenes', 'violence', 'set', 'right', 'word', 'GO.', 'Trust', 'show', 'faint', 'hearted', 'timid.', 'This', 'show', 'pulls', 'punches', 'regards', 'drugs', 'sex', 'violence.', 'Its', 'hardcore', 'classic', 'use', 'word.', 'It', 'called', 'OZ', 'nickname', 'given', 'Oswald', 'Maximum', 'Security', 'State', 'Penitentary.', 'It', 'focuses', 'mainly', 'Emerald', 'City', 'experimental', 'section', 'prison', 'cells', 'glass', 'fronts', 'face', 'inwards', 'privacy', 'high', 'agenda.', 'Em', 'City', 'home', 'many..Aryans', 'Muslims', 'gangstas', 'Latinos', 'Christians', 'Italians', 'Irish', 'more....so', 'scuffles', 'death', 'stares', 'dodgy', 'dealings', 'shady', 'agreements', 'never', 'far', 'away.', 'I', 'would', 'say', 'main', 'appeal', 'show', 'due', 'fact', 'goes', 'shows', 'dare.', 'Forget', 'pr

In [ ]:
# def remove_non_alpha(text):
#     regex = re.compile("[^a-zA-Z1-9.']")
#     return [regex.sub('',w) for w in text] 

# df['review_only_alpha'] = df['review_lemmatize'].apply(remove_non_alpha)
# print(df['review_lemmatize'][0])
# print(df['review_only_alpha'][0])

# Experiment


## Split 


In [ ]:

X = df['review_lemmatize']
y = df['sentiment_binary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

22793    [The, horse, indeed, fine, animal., Picturesqu...
33584    [And, I, do., Peter, Falk, created, role, live...
27739    [Spanish, film, Golden, definitely, Silver, Ag...
38163    [The, often, misunderstood, Zabriskie, Point, ...
23100    [This, got, one, magnificent, thing, I've, eve...
Name: review_lemmatize, dtype: object

## Deeplearning


In [ ]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(df['review_lemmatize'])

vocab_size = len(tokenizer_obj.word_index)

In [ ]:
# tokens
X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

# pad sequence 
length = max([len(s) for s in df['review_lemmatize']])
X_train_seq = pad_sequences(X_train_tokens, maxlen=length, padding='post')
X_test_seq = pad_sequences(X_test_tokens, maxlen=length, padding='post')

print(X_train_tokens[0])
print(X_train_seq[0])

[2, 1292, 846, 438, 14340, 10230, 2141, 1269, 1292, 1540, 29, 42, 12167, 787, 1657, 2, 787, 242, 62116, 2, 438, 787, 708, 13865, 882, 1292, 122905, 534, 1657, 1609, 18006, 28330, 6462, 14113, 1241, 534, 4215, 2733, 18183, 722, 4822, 2, 333, 11961, 143, 7247, 1041, 47, 136, 45, 19, 1283, 335, 18802, 1292, 9, 3910, 22, 2, 19, 438, 1292, 174, 335, 18802, 1283, 9, 205, 3, 607, 54, 15733, 5, 54, 787, 414, 5708, 14, 438, 876, 2742, 2035, 688, 700, 377, 36, 9]
[   2 1292  846 ...    0    0    0]


In [ ]:
embed_dim = 100
window = 5 
workers = 4
model = gensim.models.Word2Vec(sentences=df['review_lemmatize'], size=embed_dim, window=window, workers=workers,min_count=1)
vocab_model = model.wv.vocab

In [ ]:
model.wv.most_similar('good')

[('decent', 0.7993804216384888),
 ('great', 0.7634856700897217),
 ('nice', 0.7180683016777039),
 ('good.', 0.7103123664855957),
 ('fine', 0.6992824077606201),
 ('bad', 0.6693556904792786),
 ('excellent', 0.6501373648643494),
 ('okay', 0.6463180780410767),
 ('alright', 0.6403177976608276),
 ('cool', 0.614944577217102)]

In [ ]:
embed_idx = {}
i = 0
for word in vocab_model : 
  embed_idx[word] = model.wv.vectors[i]
  i+=1

# embed_idx

In [ ]:
embed_matrix = np.zeros((vocab_size+1,embed_dim))

for word, idx in tokenizer_obj.word_index.items() : 
  if i > (vocab_size+1) :
    continue 
  if embed_idx[word] is not None : 
    embed_matrix[i] = embed_idx[word]

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding 
from keras.initializers import Constant 

model = Sequential()
embedding_layer = Embedding(vocab_size+1, embed_dim, embeddings_initializer=Constant(embed_matrix), input_length=length, trainable=False)
model.add(embedding_layer) 
model.add(LSTM(units=32, dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1519, 100)         16737200  
                                                                 
 lstm (LSTM)                 (None, 32)                17024     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 16,754,257
Trainable params: 17,057
Non-trainable params: 16,737,200
_________________________________________________________________


In [ ]:
# X_train_seq

model.fit(X_train_seq, y_train, batch_size=100, epochs=25, validation_data=(X_test_seq,y_test))

## Naivebayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics 

#tfidf
tfidf = TfidfVectorizer(stop_words='english')
temp = df['review'].apply(lambda x: remove_tags(x))
temp_lemmatize = temp.apply(lambda x : WordNetLemmatizer().lemmatize(x,'v'))
# print(temp)
X = tfidf.fit_transform(temp_lemmatize)
vocabulary = tfidf.get_feature_names() 
# X1 = pd.DataFrame(data=X.toarray(), columns= vocabulary)
# print(X[0].dtypes) 
# print(X1)
y = df['sentiment_binary']

# print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

mnb = MultinomialNB() 
mnb.fit(X_train,y_train)

predicted_mnb = mnb.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted_mnb, y_test)
accuracy_score


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


0.867

In [ ]:
# tfidf
cv = CountVectorizer()
# X = cv.fit_transform(df['review'])
X = cv.fit_transform(temp_lemmatize)
y = df['sentiment_binary']

# print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

mnb = MultinomialNB() 
mnb.fit(X_train,y_train)

predicted_mnb = mnb.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted_mnb, y_test)
accuracy_score


0.842

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(temp_lemmatize)
y = df['sentiment_binary']

# print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

LR = LogisticRegression()
LR.fit(X_train,y_train)

predicted_lr = LR.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted_lr, y_test)
accuracy_score

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.8815

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
X = tfidf.fit_transform(temp)
y = df['sentiment_binary']

# print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

LR = LogisticRegression()
LR.fit(X_train,y_train)

predicted_lr = LR.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted_lr, y_test)
accuracy_score

0.8982

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
X = tfidf.fit_transform(temp_lemmatize)
y = df['sentiment_binary']

# print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

LR = LogisticRegression()
LR.fit(X_train,y_train)

predicted_lr = LR.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted_lr, y_test)
accuracy_score

0.8938